### 계산식 뽑는 코드
### 11,12,13,16번 보고서코드 계산식

In [4]:
import json
import csv
import re
from openai import OpenAI
from collections import defaultdict
import pandas as pd


client = OpenAI(api_key="")

# 엑셀파일 업로드
def load_financial_items_from_excel(xlsx_path, 항목명_여제목='계정명', 코드_여제목='계정코드', 년도='20231231 K 백만원'):
   df = pd.read_excel(xlsx_path, dtype=str)
   df = df.dropna(subset=[항목명_여제목, 코드_여제목, 년도])


   item_dict = {}
   for _, row in df[[코드_여제목, 항목명_여제목]].iterrows():
       full_path = row[항목명_여제목]
       item_dict[row[코드_여제목]] = full_path.strip()
   return item_dict

# 계층구조 생성
def build_hierarchy(item_dict):
   hierarchy = defaultdict(list)
   for code, path in item_dict.items():
       parts = path.split('|')
       for i in range(len(parts) - 1):
           parent = parts[i].strip()
           child = parts[i + 1].strip()
           hierarchy[parent].append(child)
   for key in hierarchy:
       hierarchy[key] = sorted(set(hierarchy[key]))
   return hierarchy

# 이름과 코드 매핑
def build_name_to_code_map(item_dict):
   name_map = defaultdict(list)
   for code, path in item_dict.items():
       parts = path.split('|')
       for i in range(len(parts)):
           name = parts[i].strip()
           parent = parts[i - 1].strip() if i > 0 else None
           if parent:
               name_map[(name, parent)].append((path, code))
               normalized = re.sub(r'[\(\)\-\s]', '', name)
               if normalized != name:
                   name_map[(normalized, parent)].append((path, code))
   return name_map


def format_prompt_from_hierarchy(hierarchy, item_dict):
    # 계정명과 코드 매핑 안내
    prompt = (
        "다음은 나이스 계정명을 포함한 재무제표 계정들의 계층 구조야.\n\n"
        "- 각 줄은 상위 항목(부모 코드 및 이름)과 그 하위 구성 항목들(자식 코드 및 이름)을 나타내고 있어.\n"
        "- 아래 정보를 참고해서 **계산이 필요한 모든 계정명에 대해 빠지지 않고 계산식과 설명을 작성해줘.**\n"
        "- 계산식은 '- **계산식**: A = B + C' 또는 '- **계산식**: A = B - C' 또는 '- **계산식**: A = B - C + D'형식으로, 설명은 '- **설명**:' 형식으로 작성해.\n"
        "- 다 더하기는 아니고 빼기도 있어. 계산식은 더하기, 빼기 둘 다 가능해. 이걸 꼭 기억해.\n"
        "- 이 규칙을 참고해서 계산식 작성 시 올바른 부호를 사용해줘.\n"
        " - 너는 이름만 보고도 차감 대상인지 추론할 수 있도록 학습되어 있으니, 합계에서 빠져야 하는 항목은 꼭 마이너스로 표시해줘.\n"
        "- 마지막 줄에 코드 계산식도 추가해. 예: '- **계산식(코드)**: 114000 = 112634 + 113200'\n"
        "- 계정명과 계정코드 매핑 정보도 같이 줄게. 이걸 활용해서 코드 치환도 해줘.\n"
        "- 같은 항목이 여러 곳에서 등장해도 중복 없이 한 번만 작성해줘.\n"
        "- 각 항목마다 반드시 **설명도 빠짐없이 작성해야 해.**\n"
        "- 설명은 '- **설명**:' 형식으로, 최대한 간단하고 명확하게 써줘.\n"
        "- 예시:\n"
        "- **계정명: 자산총계**\n"
        "- **계산식**: 자산총계 = 유동자산(계) + 비유동자산(계)\n"
        "- **설명**: 자산총계는 유동자산과 비유동자산을 합한 값입니다.\n"
        "- **계산식(코드)**: 115000 = 112000 + 114000\n\n"
        "- 절대로 계정명에 괄호를 붙이지 마세요. \n"
        " - '매출채권및기타채권 (비유동자산)' →  틀림 \n"
        " - '매출채권및기타채권' → 정답 \n"
        " - 같은 이름의 계정명이 여러 위치(예: 유동자산/비유동자산)에 등장해도 절대 괄호 등으로 구분하지 마. \n"
        " - 절대로 GPT가 알아서 설명을 붙이거나 괄호를 붙이면 안 됨. 계정명은 데이터에 있는 것 그대로 써야 함. \n"
        "- GPT 너는 계정명을 **있는 그대로만** 써야 해. 단어 하나라도 바꾸면 다른 항목으로 간주돼.\n"
        "- 괄호로 구분하거나, '계정명 (소분류)' 식으로 설명 달면 무조건 틀린 거야.\n"
        "- 예: '기타비금융부채'는 그대로 써야 해. '기타비금융부채(비유동부채)'는 틀린 이름이야.\n"
        "- 중간 항목도 계산식이 필요하니 leaf만 쓰지 말고 중간도 작성해줘.\n"
        "- '감소(증가)', '증가(감소)', '지급' 등의 표현이 들어간 항목이라도 무조건 빼는 것이 아님.\n"
        "- 모든 항목이 '+'로 계산되는 구조일 수 있으니 절대로 항목에 따라 임의로 '-'를 붙이지 마\n."
        "- 예: 퇴직금의지급, 감소(증가) 항목들도 '+'로 포함해.\n"
        "- 계정명 뒤에 (-)가 붙어있는 경우는 무조건 빼는 항목이야. 예: 주식발행초과금 + 자기주식(-) + 기타의기타자본구성 = 118214 - 118430 + 118599\n"
        "- 계정명 뒤에 (-)가 붙어있는 계정이 맨 앞에 나오더라도 마이너스 값으로 표시를 해야해. 예: '투자자산평가손실(-) + 투자자산평가이익 = -118530 + 118540\n"
        "- 최대한 계산 할 수 있는 모든 항목을 다 뽑아내야해. 꼭 빠짐없이 작성해줘.\n"
    )

    # 매핑 정보 출력
    prompt += "\n[계정명과 코드 매핑]\n"
    for code, path in item_dict.items():
        last_name = path.split('|')[-1]
        prompt += f"{last_name}: {code}\n"
        depth = path.count('|')
        prompt += f"{last_name}: {code} (depth {depth})\n"

    # 계층 구조 출력 (코드와 함께)
    prompt += "\n[계정 계층 구조 정보]\n"
    code_to_name = {code: path.split('|')[-1] for code, path in item_dict.items()}
    for parent_name, children_names in hierarchy.items():
        parent_candidates = [(code, path) for code, path in item_dict.items() if path.endswith(f"|{parent_name}") or path == parent_name]
        for parent_code, _ in parent_candidates:
            parent_full_path = item_dict[parent_code]
            prefix = parent_full_path + '|'
            child_codes = [
                code for code, path in item_dict.items()
                if path.startswith(prefix) and path.count('|') == parent_full_path.count('|') + 1
            ]
            child_descs = [f"{code} ({code_to_name.get(code, '?')})" for code in child_codes]
            line = f"{parent_code} ({parent_name}) 항목은 다음 구성 항목들과 연결되어 있음: {', '.join(child_descs)}"
            prompt += line + "\n"

    # 여기 추가됨
    prompt += (
        "\n 매우 중요 \n"
        "- GPT는 절대로 계정명에 괄호나 부연 설명을 붙이면 안 된다.\n"
        "- 반드시 원본 계정명을 그대로 사용할 것. 예: '기타비금융자산' (O), '기타비금융자산 (비유동자산)' (X)\n"
        "- 원본계정에 비유동자산(계)라고 쓰여있으면, 그대로 비유동자산(계) 라고 써. 정말 원본에 있는 계정명 그대로 가져와."
        "- 위 규칙을 어기면 전부 틀린 계산식으로 간주된다. 반드시 지켜라."

    )

    return prompt

def get_gpt_response(prompt):
   response = client.chat.completions.create(
       model="gpt-4o",
       messages=[
           {"role": "system", "content": "너는 회계와 재무제표 계산에 특화된 전문가야. 특히, 너는 나이스 평가정보에 대해 회계 지식이 풍부해."},
           {"role": "user", "content": prompt}
       ],
       temperature=0.0,
       top_p=0.0
   )
   return response.choices[0].message.content

#저장 형식 지정
def parse_markdown_to_rows(text, name_to_code_map, item_dict):
    rows = []
    current_target = None
    current_formula = None
    current_desc = None
    current_formula_code = None
    current_code_from_formula = None

    lines = text.strip().splitlines()
    for idx, line in enumerate(lines):
        line = line.strip()

        if "**계정명:" in line:
            # 이전 항목 저장
            if current_target and current_formula:
                target_code = current_code_from_formula or 'N/A'
                rows.append([
                    current_target,
                    target_code,
                    current_formula,
                    current_formula_code or current_formula,
                    current_desc or ""
                ])
            current_target = re.sub(r"^\-?\s*\*\*계정명:\s*", "", line).replace("**", "").strip()
            current_formula = None
            current_formula_code = None
            current_desc = None
            current_code_from_formula = None
        elif line.startswith("### "):  # ✅ 추가된 조건
            if current_target and current_formula:
                target_code = current_code_from_formula or 'N/A'
                rows.append([
                    current_target,
                    target_code,
                    current_formula,
                    current_formula_code or current_formula,
                    current_desc or ""
                ])
            current_target = line.replace("###", "").strip()
            current_formula = None
            current_formula_code = None
            current_desc = None
            current_code_from_formula = None

        elif "**계산식(코드)**:" in line:
            current_formula_code = re.sub(r"^\-?\s*\*\*계산식\(코드\)\*\*:\s*", "", line).strip()
            code_match = re.match(r"^(\d+)\s*=", current_formula_code)
            if code_match:
                current_code_from_formula = code_match.group(1)

        elif "**계산식**:" in line:
            current_formula = re.sub(r"^\-?\s*\*\*계산식\*\*:\s*", "", line).strip()

        elif "**설명**:" in line:
            current_desc = re.sub(r"^\-?\s*\*\*설명\*\*:\s*", "", line).strip()

    # 마지막 항목 저장
    if current_target and current_formula:
        target_code = current_code_from_formula or 'N/A'
        rows.append([
            current_target,
            target_code,
            current_formula,
            current_formula_code or current_formula,
            current_desc or ""
        ])

    return rows

def build_full_row(target_name, formula, desc, name_to_code_map, item_dict):

    # 1. 좌변 코드 먼저 추출
    code_match = re.match(r'^(\d+)\s*=', formula)
    target_code = code_match.group(1) if code_match else 'N/A'

    # ✅ 2. 코드 기반으로 전체 경로 가져옴
    target_path = item_dict.get(target_code, None)
    if not target_path:
        return [target_name, target_code, formula, formula, desc or ""]

    parent_path_parts = target_path.split('|')[:-1]
    formula_code = formula

    # 3. 우변 항목을 치환하되, 계층(parent path)이 정확히 같은 애들만
    for name, candidates in sorted(name_to_code_map.items(), key=lambda x: -len(x[0])):
        if name not in formula_code:
            continue

        for full_path, code in candidates:
            full_parts = full_path.split('|')[:-1]

            # 정확히 같은 부모 계층일 때만 코드 치환 허용
            if full_parts == parent_path_parts:
                pattern = re.compile(re.escape(name) + r"(?![\w\)])")
                formula_code = pattern.sub(code, formula_code)
                break  # 첫 일치만

    # 4. 좌변도 코드로 고정
    formula_code = re.sub(r'^.*?=', f"{target_code} =", formula_code)

    return [target_name, target_code, formula, formula_code, desc or ""]


def save_to_csv(rows, file_path='/Users/bag-yunji/Desktop/NICE/정만이/윤지/samsung/2023계산식/samsung_2023_16.csv'):
   with open(file_path, 'w', newline='', encoding='utf-8-sig') as f:
       writer = csv.writer(f)
       writer.writerow(['계산 대상', '계정 코드', '계산식', '계산식 코드', '설명'])
       writer.writerows(rows)


# 실행
if __name__ == "__main__":
   excel_path = "/Users/bag-yunji/Desktop/NICE/정만이/윤지/samsung/answer/samsung_16_2023.xlsx"
   items = load_financial_items_from_excel(excel_path)
   hierarchy = build_hierarchy(items)
   name_to_code_map = build_name_to_code_map(items)
   prompt = format_prompt_from_hierarchy(hierarchy, items)


   gpt_answer = get_gpt_response(prompt)
   rows = parse_markdown_to_rows(gpt_answer, name_to_code_map, items)


   print("🧾 GPT 응답 ↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓")
   print(gpt_answer)


   if rows:
       save_to_csv(rows)
       print(f"저장 완료: 16.csv (총 {len(rows)}행)")
   else:
       print("파싱 실패: GPT 응답 포맷을 확인하세요.")



🧾 GPT 응답 ↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓
- **계정명: 영업활동으로인한현금흐름**
- **계산식**: 영업활동으로인한현금흐름 = 당기순이익(손실) + 현금유출없는비용등가산 - 현금유입없는수익등차감 + 영업활동관련자산부채변동 - 금융비용유출액 + 이자수익유입액 + 배당금수익유입액 - 법인세의납부(환급)
- **설명**: 영업활동으로인한현금흐름은 당기순이익에 비현금성 비용을 더하고 비현금성 수익을 차감하며, 자산 및 부채의 변동을 반영하여 계산됩니다.
- **계산식(코드)**: 161000 = 161100 + 161210 - 161510 + 161540 - 161710 + 161711 + 161713 - 161714

- **계정명: 현금유출없는비용등가산**
- **계산식**: 현금유출없는비용등가산 = 유형자산, 투자부동산감가상각비 + 무형자산상각비 + 대손상각비 + 퇴직급여 + 유,무형,리스자산처분손실 + 재고자산평가(감모)손실 + 이자비용 + 현금의유출이없는기타비용
- **설명**: 현금유출없는비용등가산은 비현금성 비용을 합산하여 계산됩니다.
- **계산식(코드)**: 161210 = 161211 + 161212 + 161214 + 161215 + 161219 + 161221 + 161226 + 161239

- **계정명: 무형자산상각비**
- **계산식**: 무형자산상각비 = 개발비상각 + 기타무형자산상각비
- **설명**: 무형자산상각비는 개발비와 기타 무형자산의 상각비를 합산하여 계산됩니다.
- **계산식(코드)**: 161212 = 161242 + 161249

- **계정명: 현금유입없는수익등차감**
- **계산식**: 현금유입없는수익등차감 = 유,무형,리스자산처분이익 + 이자수익 + 배당금수익 + 법인세비용의환급
- **설명**: 현금유입없는수익등차감은 비현금성 수익을 합산하여 계산됩니다.
- **계산식(코드)**: 161510 = 161514 + 161521 + 161555 + 161556

- **계정명: 영업

### 추출된 계산식 기반으로 정답지에서 값을 불러와 계산 진행

In [1]:
import pandas as pd
import csv
import re

# 경로 설정
formula_path = "/Users/bag-yunji/Desktop/NICE/정만이/윤지/kolon/2023계산식/kolon_2023_11_final_FINAL_GOD.csv"  # 계산식 파일
value_path = "/Users/bag-yunji/Desktop/NICE/정만이/윤지/kolon/32_500461_코오롱글로벌(주)_개별.xls"
output_path = "/Users/bag-yunji/Desktop/NICE/정만이/윤지/kolon/2023계산식/kolon_11_2023_calc.csv"  # 결과 저장 경로

# 1. 숫자 값 불러오기
df = pd.read_excel(value_path, dtype=str)
df = df.dropna(subset=['계정코드', '20231231 K 원'])

# 계정코드 → 숫자 값 딕셔너리
value_dict = {
    str(row['계정코드']).strip(): float(str(row['20231231 K 원']).replace(',', ''))
    for _, row in df.iterrows()
}

# 2. 수식 파일 불러오기
formulas = []
with open(formula_path, 'r', encoding='utf-8-sig') as f:
    reader = csv.reader(f)
    next(reader)  # header skip
    for row in reader:
        if len(row) >= 5:
            target_name, target_code, formula, formula_code, desc = row
            formulas.append((target_name, target_code.strip(), formula, formula_code.strip(), desc))

# 3. 계산 수행
def is_valid_formula_code(code_str):
    """수식이 숫자와 연산자만 포함되어 있는지 확인"""
    return bool(re.fullmatch(r'[\d\s\+\-\*/\(\)]+', code_str))

results = []
for target_name, target_code, formula, formula_code, desc in formulas:
    eval_formula = formula_code

    # 등호가 있으면 우변만 남김
    if '=' in eval_formula:
        eval_formula = eval_formula.split('=', 1)[1].strip()

    # 수식 유효성 확인
    if not eval_formula or not is_valid_formula_code(eval_formula):
        print(f"❌디버깅] 유효하지 않은 formula_code: {eval_formula}")
        results.append([target_name, target_code, formula, formula_code, desc, "오류: 잘못된 계산식 형식"])
        continue

    # 코드 치환: 6자리 숫자를 value_dict[...]로 변경
    codes_in_formula = re.findall(r'\b\d{6}\b', eval_formula)
    for code in sorted(set(codes_in_formula), key=len, reverse=True):
        if code in value_dict:
            eval_formula = eval_formula.replace(code, f'value_dict["{code}"]')
        else:
            eval_formula = eval_formula.replace(code, '0')  # 없으면 0으로 처리

    # 디버깅 출력
    print(f"\n계산 대상: {target_name} ({target_code})")
    print(f" - 원래 수식: {formula_code}")
    print(f" - eval 수식: {eval_formula}")
    print(" - 코드별 값:")
    for code in codes_in_formula:
        print(f"   · {code}: {value_dict.get(code, '없음')}")

    try:
        result = eval(eval_formula)
        print(f" 계산 결과: {round(result)}")
        results.append([target_name, target_code, formula, formula_code, desc, round(result)])
    except Exception as e:
        print(f" 계산 오류: {e}")
        results.append([target_name, target_code, formula, formula_code, desc, f"오류: {e}"])

# 4. 결과 저장
with open(output_path, 'w', newline='', encoding='utf-8-sig') as f:
    writer = csv.writer(f)
    writer.writerow(['계정명', '계정코드', '계산식', '계산식(코드)', '설명', '계산 결과'])
    writer.writerows(results)

print(f"\n계산 완료 및 저장: {output_path}")


📌 계산 대상: 자산총계 (115000)
 - 원래 수식: 115000 = 114000 +112000
 - eval 수식: value_dict["114000"] +value_dict["112000"]
 - 코드별 값:
   · 114000: 1064098723007.0
   · 112000: 1454449333081.0
 ✅ 계산 결과: 2518548056088

📌 계산 대상: 비유동자산(계) (114000)
 - 원래 수식: 114000 = 113200 +112150 +113400 +113383 +112631 +112270 +112634 +112636
 - eval 수식: value_dict["113200"] +value_dict["112150"] +value_dict["113400"] +value_dict["113383"] +value_dict["112631"] +value_dict["112270"] +0 +value_dict["112636"]
 - 코드별 값:
   · 113200: 281153342226.0
   · 112150: 421689820106.0
   · 113400: 13248281315.0
   · 113383: 154886908424.0
   · 112631: 106324591263.0
   · 112270: 85479780273.0
   · 112634: ❌ 없음
   · 112636: 1315999400.0
 ✅ 계산 결과: 1064098723007

📌 계산 대상: 유형자산(계) (113200)
 - 원래 수식: 113200 = 113310 +113110 +113120 +113130 +113140 +113160 +113170 +113180 +113199
 - eval 수식: value_dict["113310"] +value_dict["113110"] +value_dict["113120"] +value_dict["113130"] +value_dict["113140"] +value_dict["113160"] +value_dict["